In [17]:
import numpy as np
from collections import Counter


class BM25_Model(object):
    def __init__(self, documents_list, k1=1.25, b=0.75):
        self.documents_list = documents_list
        self.documents_number = len(documents_list)
        self.avg_documents_len = sum([len(document) for document in documents_list]) / self.documents_number
        self.f = []
        self.idf = {}
        self.k1 = k1
        self.b = b
        self.init()

    def init(self):
        df = {}
        for document in self.documents_list:
            temp = {}
            for word in document:
                temp[word] = temp.get(word, 0) + 1
            self.f.append(temp)
            for key in temp.keys():
                df[key] = df.get(key, 0) + 1
        for key, value in df.items():
            self.idf[key] = np.log((self.documents_number - value + 0.5) / (value + 0.5)+1)

    def get_score(self, index, query):
        score = 0.0
        document_len = len(self.f[index])
        qf = Counter(query)
        for q in query:
            if q not in self.f[index]:
                continue
            score += self.idf[q] * (self.f[index][q] * (self.k1 + 1) / (
                        self.f[index][q] + self.k1 * (1 - self.b + self.b * document_len / self.avg_documents_len)))

        return score

    def get_documents_score(self, query):
        score_list = []
        for i in range(self.documents_number):
            score_list.append(self.get_score(i, query))
        return score_list

    def rank_documents(self, query):
        document_scores = [(i+1, score) for i, score in enumerate(self.get_documents_score(query))]
        ranked_documents = sorted(document_scores, key=lambda x: x[1], reverse=True)
        return ranked_documents

# 示例文档列表
documents_list = [
    ["apple", "banana", "orange", "apple"],
    ["orange", "kiwi", "banana", "kiwi"],
    ["apple", "banana", "kiwi"]
]

# 创建BM25模型对象
bm25_model = BM25_Model(documents_list)

# 要查询的查询字符串
query = ["apple", "kiwi"]

# 获取每个文档的BM25分数
scores = bm25_model.get_documents_score(query)

# 将文档和分数一一对应打印出来
for i, score in enumerate(scores):
    print(f"Document {i + 1}: BM25 Score = {score}")

# 根据分数对文档进行排序
ranked_documents = bm25_model.rank_documents(query)
print("Ranked Documents:", ranked_documents)
top_document, top_score = ranked_documents[0]
print(f"Top Document: {top_document}, Top Score: {top_score}")

Document 1: BM25 Score = 0.6867949711487502
Document 2: BM25 Score = 0.6867949711487502
Document 3: BM25 Score = 1.0170570337776572
Ranked Documents: [(3, 1.0170570337776572), (1, 0.6867949711487502), (2, 0.6867949711487502)]
Top Document: 3, Top Score: 1.0170570337776572
